# 0. 환경 설정 필요 라이브러리 import

In [ ]:
import os
import time
import yaml
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import librosa
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import roc_curve, auc


### 연산 위치 저장

- 로컬 컴퓨터 맥북, mps 출력 확인

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Device:", device)

# 1.Cofing

### 실험에 필요한 변수 저장

In [ ]:
cfg = {
    "data": {
        "root": "/Users/taehayeong/Desktop/data/deepvoice_data/data",
        # 데이터 파일 경로
        "sample_rate": 16000,
        # 사람 음성 정보 가장 밀집된 대역, 일반적으로 16khz
        "n_mels": 80,
        "segment_frames": 300,
        "hop_length": 256,
    },
    "model": {
        "hidden_dim": 128,
        "latent_dim": 64,
        "num_layers": 2,
    },
    "training": {
        "batch_size": 8,
        "lr": 0.001,
        "epochs": 20,
    }
}


# 2. Data Split

### model에 따른 data 분할 방식 설정.

- detection 모델 : trian = 오직 정상, test = 정상 + 딥보이스
- classification 모델 : train , test = 정상 + 딥보이스

In [ ]:
def get_data(model="anomaly", root=None, seed=42):
    random.seed(seed)
    #실험 데이터 고정
    speaker_dirs = [
        os.path.join(root, d)
        for d in os.listdir(root)
        if d.isdigit() and os.path.isdir(os.path.join(root, d))
    ]
    #flac, 정상 파일의 폴더는 화자 단위의 번호로 되어있는 폴더. 화자 단위로 리스트 형성

    random.shuffle(speaker_dirs)
    train_speakers = speaker_dirs[:100]
    test_speakers  = speaker_dirs[100:200]
    # 화자 단위 list에서 다른 화자들로 구성.

    def collect_files(speakers):
        flacs, wavs = [], []
        for spk in speakers:
            for r, _, files in os.walk(spk):
                for f in files:
                    if f.endswith(".flac"):
                        flacs.append(os.path.join(r, f))
        wav_root = os.path.join(root, "wavs")
        if os.path.isdir(wav_root):
            for r, _, files in os.walk(wav_root):
                for f in files:
                    if f.endswith(".wav"):
                        wavs.append(os.path.join(r, f))
        return flacs, wavs
    #flac은 정상, wavs는 비정상 파일로 구분

    train_flac, train_wav = collect_files(train_speakers)
    test_flac,  test_wav  = collect_files(test_speakers)

    if model == "anomaly":
        train = train_flac
        test = [(p, 0) for p in test_flac] + [(p, 1) for p in test_wav]
        random.shuffle(test)
        return train, test
    #detect 모델은 train에는 정상만, test는 비정상만 포함 & 라벨 부여.
    elif model == "classifier":
        train = [(p, 0) for p in train_flac] + [(p, 1) for p in train_wav]
        test  = [(p, 0) for p in test_flac]  + [(p, 1) for p in test_wav]
        random.shuffle(train)
        random.shuffle(test)
        n = len(train)
        n_val = int(0.2 * n)
        return train[:-n_val], train[-n_val:], test
    #classifier 모델은 train test 둘다 정상 비정상 데이터 포함.


# 3. Preprocessor & extract Feature

In [ ]:
class AudioPreprocessor:
    def __init__(self, sr):
        self.sr = sr
    #config에서 저장한 samplate 받기
    def preprocess(self, path):
        wav, sr = librosa.load(path, sr=self.sr)
        #wav는 1차원 numpy배열
        return wav / (np.max(np.abs(wav)) + 1e-9)
    #음성 파일 읽고, 파형으로 변환후 절대값을 통해 크기 확인,
    #max를 통해 파형 전체를 가장 큰 값으로 나눔 -> [-1,1] 정규화
    #소리 크기가 아닌 목소리 특징 학습을 위해 필요.
    #1e+9로 0 방지

### log-mel spectro

- sample rate(sr)는 초당 소리를 몇 칸으로 잘라서 저장하느냐이다. 즉, sr = 16000이라면, 1초 동안 소리를 16000번 찍는것이다.
- wav : 파형은 소리가 흔들린 기록이다. 이 wav를 1초당 sr로 나눈다. 
- n_fft : 1초당 sr로 나눈것을 n_fft덩어리로 묶는다. 음성은 여러 시점에서의 소리를 합쳐 인식되기 때문에 필요.
- hop_length : hop을 통해 하나의 음성에서 여러 분할로 소리가 시간에 따라 어떻게 변하는지 확인.
- n_mels : 주파수 영역을 n_mels값으로 분할한것.
- log : 소리 에너지를 log를 통해 작은 에너지도 확인 가능하게 한다.

#### n_fft를 통해 주파수 분석의 해상도를, n_mels를 통해 사람이 듣는 방식으로 요약.

In [ ]:
def extract_logmel(wav, sr, n_mels):
    mel = librosa.feature.melspectrogram(
        y=wav,
        sr=sr,
        n_fft=1024,
        hop_length=256,
        n_mels=n_mels
    )
    return librosa.power_to_db(mel)

# 4. Dataset

실제 모델 입력 데이터 변환

In [ ]:
class VoiceDataset(Dataset):
    def __init__(self, file_list, preprocessor, feature_fn, segment_len, mode):
        self.items = []
        self.wav_cache = {}
        #wav 한번만 읽기위해 사용
        self.feature_fn = feature_fn
        self.segment_len = segment_len
        self.mode = mode

        for wid, item in enumerate(file_list):
            if mode == "anomaly_train":
                path, label = item, 0
            else:
                path, label = item

            wav = preprocessor.preprocess(path)
            self.wav_cache[path] = wav

            n_seg = len(wav) // segment_len
            #통화 초반 몇 초를 여러 조각으로 나눔
            for i in range(n_seg):
                self.items.append((path, label, i, wid))
            #segment 단위 item 생성
    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, label, seg_idx, wid = self.items[idx]
        wav = self.wav_cache[path]
        seg = wav[seg_idx*self.segment_len:(seg_idx+1)*self.segment_len]
        feat = self.feature_fn(seg)
        #모델 입력 형태 (n_mes, T) T 는 시간 축.

        if self.mode == "anomaly_train":
            return feat
        elif "test" in self.mode:
            return feat, label, wid
        else:
            return feat, label


# 5. Model

## 5.1 LSTM + AE

- (1):init
    - 모델 입력 변수 초기화
- (2):encoder
    - 음성 특징 학습
- (3):latent
    - encoder 결과 요약
- (4):decoder
    - 요약 결과 기존 형태 복원

In [ ]:
class LSTMAutoEncoder(nn.Module):
    def __init__(
        self,
        n_mels=80,
        hidden_dim=128,
        latent_dim=64,
        num_layers=2
    ):
        super().__init__()

        # -------- Encoder --------
        self.encoder = nn.LSTM(
            input_size=n_mels,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )

        self.to_latent = nn.Linear(hidden_dim, latent_dim)

        # -------- Decoder --------
        self.from_latent = nn.Linear(latent_dim, hidden_dim)

        self.decoder = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=n_mels,
            num_layers=num_layers,
            batch_first=True
        )

    def forward(self, x):
        """
        x: (B, n_mels, T)
        return: reconstructed x (B, n_mels, T)
        """

        x = x.permute(0, 2, 1)

        # -------- Encoder --------
        enc_out, (h_n, _) = self.encoder(x)

        h_last = h_n[-1]                 # (B, hidden_dim)
        z = self.to_latent(h_last)       # (B, latent_dim)

        # -------- Decoder --------
        h_dec = self.from_latent(z)      # (B, hidden_dim)
        T = x.size(1)
        h_dec_seq = h_dec.unsqueeze(1).repeat(1, T, 1)

        recon, _ = self.decoder(h_dec_seq)
        recon = recon.permute(0, 2, 1)

        return recon

## 5.2 CNN + BiLSTM

- (1) CNN : 주파수 - 시간 국소 패턴 추출
- (2) BiLSTM : 발화 시간적 흐름 학습

In [ ]:
class DeepVoiceClassifier(nn.Module):
    def __init__(
        self,
        n_mels=80,
        lstm_hidden=128,
        num_classes=2
    ):
        super().__init__()

        # -------- CNN Encoder --------
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        # -------- BiLSTM --------
        self.lstm = nn.LSTM(
            input_size=(n_mels // 4) * 32,
            hidden_size=lstm_hidden,
            batch_first=True,
            bidirectional=True
        )

        # -------- Classifier --------
        self.fc = nn.Linear(lstm_hidden * 2, num_classes)

    def forward(self, x):
        """
        x: (B, 1, n_mels, T)
        """
        
        x = self.conv(x)

        B, C, F, T = x.shape
        x = x.permute(0, 3, 1, 2)  # (B, T, C, F)
        x = x.reshape(B, T, C * F) # (B, T, feature)

        out, _ = self.lstm(x)

        out = out[:, -1, :]

        logits = self.fc(out)
        return logits

---

# Plot Roc

In [ ]:
def plot_roc(scores, labels, title="ROC"):
    fpr, tpr, _ = roc_curve(labels, scores)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1],'--')
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()
    return roc_auc


# 6.Train LSTM + AE

In [ ]:
train_files, _ = get_data("anomaly", cfg["data"]["root"])
prep = AudioPreprocessor(cfg["data"]["sample_rate"])

ae_dataset = VoiceDataset(
    train_files, prep,
    lambda x: torch.tensor(extract_logmel(x, cfg["data"]["sample_rate"], cfg["data"]["n_mels"]), dtype=torch.float32),
    cfg["data"]["segment_frames"] * cfg["data"]["hop_length"],
    "anomaly_train"
)

ae_loader = DataLoader(ae_dataset, batch_size=cfg["training"]["batch_size"], shuffle=True)
#데이터 준비, tensor 변환

ae = LSTMAutoEncoder(cfg["data"]["n_mels"], cfg["model"]["hidden_dim"],
                     cfg["model"]["latent_dim"], cfg["model"]["num_layers"]).to(device)
#model 생성
opt = torch.optim.Adam(ae.parameters(), lr=cfg["training"]["lr"])
loss_fn = nn.MSELoss()
#optimizer & Loss function
ae_train_times = []
#training 시간 저장
for e in range(cfg["training"]["epochs"]):
    start = time.time()
    for x in ae_loader:
        x = x.to(device)
        loss = loss_fn(ae(x), x)
        opt.zero_grad()
        loss.backward()
        opt.step()
    ae_train_times.append(time.time() - start)
    print(f"[AE Epoch {e+1}] loss={loss.item():.4f}")
#training- 복원 오차 계산, 더 잘 복원하도록 가중치 업데이트.

os.makedirs("checkpoints", exist_ok=True)
AE_CKPT_PATH = "checkpoints/lstm_ae.pth"
torch.save(ae.state_dict(), AE_CKPT_PATH)
#학습된 모델 저장.
print(f"AE model saved to {AE_CKPT_PATH}")

# 7. Test LSTM + AE

In [ ]:
_, test = get_data("anomaly", cfg["data"]["root"])
prep = AudioPreprocessor(cfg["data"]["sample_rate"])

dataset = VoiceDataset(
    test,
    prep,
    lambda x: torch.tensor(
        extract_logmel(x, cfg["data"]["sample_rate"], cfg["data"]["n_mels"]),
        dtype=torch.float32
    ),
    cfg["data"]["segment_frames"] * cfg["data"]["hop_length"],
    "anomaly_test"
)

loader = DataLoader(dataset, batch_size=1, shuffle=False)

AE_CKPT_PATH = "checkpoints/lstm_ae.pth"

model = LSTMAutoEncoder(
    cfg["data"]["n_mels"],
    cfg["model"]["hidden_dim"],
    cfg["model"]["latent_dim"],
    cfg["model"]["num_layers"]
).to(device)

model.load_state_dict(torch.load(AE_CKPT_PATH, map_location=device))
model.eval()


criterion = nn.MSELoss(reduction="none")

wav_scores = defaultdict(list)
wav_labels = {}


start_test = time.time()

with torch.no_grad():
    for x, label, wid in loader:
        x = x.to(device)
        recon = model(x)
        score = criterion(recon, x).mean().item()
        wav_scores[wid.item()].append(score)
        wav_labels[wid.item()] = label.item()

total_test_time = time.time() - start_test
print("Total AE test time:", total_test_time)

final_scores, final_labels = [], []
for wid, scores in wav_scores.items():
    final_scores.append(np.mean(sorted(scores, reverse=True)[:5]))
    final_labels.append(wav_labels[wid])
#wav안 특정 구간 가장 이상한 순간만 저장.

auc = plot_roc(np.array(final_scores), np.array(final_labels))
print("AUC:", auc)

np.save("ae_test_time.npy", np.array([total_test_time]))

# 8. Train CNN + BiLSTM

In [ ]:
train, _, _ = get_data("classifier", cfg["data"]["root"])

clf_dataset = VoiceDataset(
    train, prep,
    lambda x: torch.tensor(extract_logmel(x, cfg["data"]["sample_rate"], cfg["data"]["n_mels"]), dtype=torch.float32).unsqueeze(0),
    cfg["data"]["segment_frames"] * cfg["data"]["hop_length"],
    "classifier"
)

clf_loader = DataLoader(clf_dataset, batch_size=cfg["training"]["batch_size"], shuffle=True)

clf = DeepVoiceClassifier(cfg["data"]["n_mels"], cfg["model"]["hidden_dim"]).to(device)
opt = torch.optim.Adam(clf.parameters(), lr=cfg["training"]["lr"])
loss_fn = nn.CrossEntropyLoss()

clf_train_times = []

for e in range(cfg["training"]["epochs"]):
    start = time.time()
    correct = total = 0
    for x, y in clf_loader:
        x, y = x.to(device), y.to(device)
        logits = clf(x)
        loss = loss_fn(logits, y)
        opt.zero_grad()
        loss.backward()
        opt.step()

        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    clf_train_times.append(time.time() - start)
    acc = correct / total if total > 0 else 0
    print(f"[CLF Epoch {e+1}] acc={acc:.4f}")

os.makedirs("checkpoints", exist_ok=True)
CLF_CKPT_PATH = "checkpoints/classifier.pth"
torch.save(clf.state_dict(), CLF_CKPT_PATH)

print(f"Classifier model saved to {CLF_CKPT_PATH}")



# 9.Test CNN + BiLSTM

In [ ]:
_, _, test = get_data("classifier", cfg["data"]["root"])
prep = AudioPreprocessor(cfg["data"]["sample_rate"])

dataset = VoiceDataset(
    test,
    prep,
    lambda x: torch.tensor(
        extract_logmel(x, cfg["data"]["sample_rate"], cfg["data"]["n_mels"]),
        dtype=torch.float32
    ).unsqueeze(0),
    cfg["data"]["segment_frames"] * cfg["data"]["hop_length"],
    "classifier_test"
)

loader = DataLoader(dataset, batch_size=1, shuffle=False)

CLF_CKPT_PATH = "checkpoints/classifier.pth"

model = DeepVoiceClassifier(
    cfg["data"]["n_mels"],
    cfg["model"]["hidden_dim"]
).to(device)

model.load_state_dict(torch.load(CLF_CKPT_PATH, map_location=device))
model.eval()


wav_scores = defaultdict(list)
wav_labels = {}

start_test = time.time()

with torch.no_grad():
    for x, label, wid in loader:
        x = x.to(device)
        prob = torch.softmax(model(x), dim=1)[0, 1].item()
        wav_scores[wid.item()].append(prob)
        wav_labels[wid.item()] = label.item()

total_test_time = time.time() - start_test
print("Total classifier test time:", total_test_time)

np.save("clf_test_time.npy", np.array([total_test_time]))


final_scores, final_labels = [], []
for wid, scores in wav_scores.items():
    final_scores.append(np.mean(sorted(scores, reverse=True)[:5]))
    final_labels.append(wav_labels[wid])

auc = plot_roc(np.array(final_scores), np.array(final_labels))
print("AUC:", auc)


# train & test time 비교

In [ ]:
plt.plot(ae_train_times, label="AE Train")
plt.plot(clf_train_times, label="Classifier Train")
plt.legend()
plt.show()

# Streamlit demo web

In [ ]:
import streamlit as st
import torch
import numpy as np

from src.models import LSTMAutoEncoder
from src.utils import extract_logmel
from src.preprocess import AudioPreprocessor  


def split_audio(wav, segment_len):
    n = len(wav) // segment_len
    return [wav[i*segment_len:(i+1)*segment_len] for i in range(n)]


@st.cache_resource
def load_model():
    model = LSTMAutoEncoder(
        n_mels=80,
        hidden_dim=128,
        latent_dim=64,
        num_layers=2
    )
    model.load_state_dict(
        torch.load("checkpoints/lstm_ae.pth", map_location="cpu")
    )
    #저장된 학습 모델 로드
    model.eval()
    return model


model = load_model()

st.title("Deep Voice Protector")

uploaded = st.file_uploader("통화 음성 파일 업로드 (.wav/.flac)", type=["wav", "flac"])

if uploaded and st.button("탐지 시작"):
    prep = AudioPreprocessor(sr=16000)
    wav = prep.preprocess(uploaded)

    segments = split_audio(
        wav,
        segment_len=300 * 256
    )

    errors = []

    with st.spinner("AI가 통화 초반 음성을 분석 중입니다..."):
        for seg in segments[:5]:
            feat = torch.tensor(
                extract_logmel(seg, sr=16000, n_mels = 80),
                dtype=torch.float32
            ).unsqueeze(0)

            with torch.no_grad():
                recon = model(feat)
                err = torch.mean((recon - feat) ** 2).item()
                errors.append(err)

    score = float(np.mean(errors))
    risk = min(score / 0.7, 1.0) * 100

    st.subheader("탐지 결과")
    st.write(f"딥보이스 의심 확률: **{risk:.1f}%**")

    if risk >= 70:
        st.error("⚠ 딥보이스 위험 높음\n\n송금 보류 및 재확인 권장")
    else:
        st.success("정상 음성으로 판단됨")



